# Web Scraping 


In [ ]:
!apt-get update
!apt install chromium-chromedriver
!pip install selenium

Get:1 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease [3,626 B]
Hit:2 http://archive.ubuntu.com/ubuntu bionic InRelease
Get:3 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Get:4 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Get:5 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease [15.9 kB]
Get:6 http://archive.ubuntu.com/ubuntu bionic-backports InRelease [83.3 kB]
Hit:7 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Hit:8 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu bionic InRelease
Ign:9 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Hit:10 http://ppa.launchpad.net/graphics-drivers/ppa/ubuntu bionic InRelease
Get:11 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease [1,581 B]
Hit:12 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Get:13 http://archive

In [ ]:
import time
import requests
import pandas as pd
from bs4 import BeautifulSoup
import re
import numpy as np

# Bibliotecas do selenium para pegar dados.
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC


# Gráfico
import matplotlib.pyplot as plt
import seaborn as sns
from IPython.core.display import display, HTML
import matplotlib.patheffects as path_effects

/usr/local/lib/python3.8/dist-packages/requests/__init__.py:89: RequestsDependencyWarning: urllib3 (1.26.13) or chardet (3.0.4) doesn't match a supported version!
  warnings.warn("urllib3 ({}) or chardet ({}) doesn't match a supported "


In [ ]:

# -------------- Função para coleta dos links das vagas.

def coleta(lk):
  """
  Modelo vai receber find_elements(By.TAG_NAME, 'a') da página em acesso no momento, com todas as tags
  "a" selecionadas, será separado o atributo "href" que é o link para acessar a vaga de emprego, como todos esse links
  tem mais de 200 caracters, faz-se uma condicional para poder selecionar somente os links.
  """

  lista = []
  for i in lk:
    try:
      url = i.get_attribute('href')
      if len(url) > 200:
            lista.append(url)
    except:
      continue

  return list(set(lista))



# -------------- Criação do drive para acessar o site.

chrome_options = webdriver.ChromeOptions()
chrome_options.add_argument('--headless') 
chrome_options.add_argument('--no-sandbox')
chrome_options.add_argument('--disable-dev-shm-usage')

driver = webdriver.Chrome(chrome_options=chrome_options)
urls = 'https://www.glassdoor.com.br/Vaga/brasil-cientista-de-dados-vagas-SRCH_IL.0,6_IN36_KO7,25.htm?fromAge=14'
driver.get(urls) # Criação do driver.



# -------------- Coleta de dados.


  # Identificar a quantidade de  páginas com ofertas de emprego:
i = driver.find_element(By.CLASS_NAME,'pageContainer').get_attribute('innerHTML')
button = len(BeautifulSoup(i, 'html.parser').find_all('button')) # Identifica quantos botões tem

  # Listas para armazenar os links.
listas = []

for i in range(2,button,1): # botão 1 para voltar, botão 2 da primeira página, por isso começa a partir do 2.
  # Recolher as informações da primeira página localizando pela tag "a" e aplicando na função para coletar.
  if i == 2:
    print('Página Coletada = > ',i-1)
    lk = driver.find_elements(By.TAG_NAME, 'a')
    links = coleta(lk)
    print(len(links))
    listas.extend(links) # A função trás os itens em lista, aqui faz a junção das duas.
  else:
    # Aqui é feito o click em cada página nova e recolhido os dados.
    print('Página Coletada => ',i-1)
    driver.find_element(By.XPATH, f'//*[@id="MainCol"]/div[2]/div/div[1]/button[{i}]').click()
    time.sleep(5) # Para dar tempo da página carregar as informações.
    lk = driver.find_elements(By.TAG_NAME, 'a')
    links = coleta(lk)
    print(len(links))
    listas.extend(links)
    driver.get(urls) # Para evitar o erro do site.

driver.quit() # Fechar o driver.

<ipython-input-4-7a2cf0193cdb>:30: DeprecationWarning: use options instead of chrome_options
  driver = webdriver.Chrome(chrome_options=chrome_options)


Página Coletada = >  1
30
Página Coletada =>  2
30
Página Coletada =>  3
30


In [ ]:
def head_f(x):

  p = re.compile(r'(\n)(.*)(\n)')
  r = p.finditer(x)
  lista = list()
  for i in r:
    lista.append(i.group(2))
  return lista

def outros_elementos(dt):
  """
  Função para coletar os dados (caso exista) no link de cada oferta de trabalho.
  dt = Dataset que está sendo salva as informações:
   
  """
  # ------ Parte 1 - Clicando no botão: Empresa
  try: # Caso exista esse botão o código irá rodar e coletar os dados.
    driver_1.find_element(By.XPATH, '//*[@id="PageBodyContents"]/div[2]/header/div/div/div[2]').click() # Clicar no box 
    elementos = driver_1.find_element(By.XPATH,'//*[@id="InfoFields"]').get_attribute('innerHTML') # Coletar dados da caixa
    sopa = BeautifulSoup(elementos, 'html.parser') 
    Titulo = sopa.find_all('label')
    Resposta = sopa.find_all('span')
    for t, r in zip(Titulo, Resposta):
      dt[t.text] = r.text

    try:
      # Coletar dados de avaliação da empresa
      driver_1.find_element(By.XPATH, '//*[@id="PageBodyContents"]/div[2]/header/div/div/div[3]').click() # Clickar no box
      elementos = driver_1.find_element(By.XPATH, '//*[@id="PageBodyContents"]/div[2]/div/div/div/ul').get_attribute('innerHTML') 
      sopa = BeautifulSoup(elementos, 'html.parser').find_all('span',{'class':'e121l59f1'}) # Caixa
      for i in range(0,len(sopa),2):
        dt[sopa[i].text] = sopa[i+1].text # Coletando elementos da caixa
      
      page = driver_1.page_source # Coletar todos os elementos da pag
      ele = BeautifulSoup(page, 'html.parser').find('div',{'class':'d-flex flex-column align-items-center top'}) 
      titulo = ele.find('div',{'class':'css-3jq91s e1qhxspr5'}).text
      resposta = ele.find('g').find('text').text
      dt[titulo] = resposta

    except:
      print('Failed 2')

    return dt
  except: # Caso contrário irá retornar com os dados sem qualquer alteração. 
    return dt

In [ ]:
driver_1 = webdriver.Chrome(chrome_options=chrome_options)
dados = list()
for i in range(len(listas)):
  driver_1.get(listas[i])
  
  # ----------- Localizar Nome da empresa e nota
  soup = BeautifulSoup(driver_1.page_source, 'html.parser')
  head = soup.find_all('div',{'class':'e11nt52q0'})
  head = head[0].find('div',{'class':'e11nt52q1'}).prettify()
  head = head_f(head)
  anuncio = soup.find('div', {'class':"e11nt52q6"}).text
  local = soup.find('div', {'class':"e11nt52q2"}).text
  if len(head) > 2:
    data = {'Empresa':head[0], 'Vaga':anuncio, 'Local':local,'Nota':head[1], 'Link':listas[i]}
  else:
    data = {'Empresa':head[0], 'Vaga':anuncio, 'Local':local, 'Link':listas[i]}

  print(f'Coletando dados da empresa: {data["Empresa"]}')
  

  # ----------- Localizar Descrição da vaga.
  body = soup.find_all('div',{'class':'e1eh6fgm1'})
  data['Descrição'] = body[0].find_all('div',{'id':"JobDescriptionContainer"})[0].text

  # ----------- Pegando elementos de outras páginas.
  data = outros_elementos(data)

  dados.append(data)

driver_1.quit()


<ipython-input-6-9273accb575c>:1: DeprecationWarning: use options instead of chrome_options
  driver_1 = webdriver.Chrome(chrome_options=chrome_options)


Coletando dados da empresa:  Unisoma Computação
Coletando dados da empresa:  Digio
Coletando dados da empresa:  GRUPO AGROMERCANTIL
Coletando dados da empresa:  A3Data
Coletando dados da empresa:  marketdata
Coletando dados da empresa:  Mutant
Coletando dados da empresa:  Worley
Coletando dados da empresa:  Fundação Ezute
Coletando dados da empresa:  Seazone
Coletando dados da empresa:  EY
Coletando dados da empresa:  Divulga Vagas
Coletando dados da empresa:  Propz
Coletando dados da empresa:  Mazzatech
Coletando dados da empresa:  MRO
Coletando dados da empresa:  Netvagas - (3460991214)
Coletando dados da empresa:  IBM
Coletando dados da empresa:  Zult Consultoria Ltda.
Coletando dados da empresa:  Netvagas - (3699981214)
Coletando dados da empresa:  Grupo MAG
Coletando dados da empresa:  GLOBAL HITSS
Coletando dados da empresa:  Taking
Coletando dados da empresa:  Globo
Coletando dados da empresa:  Cielo
Coletando dados da empresa:  PeoplePro Informática
Coletando dados da empresa: 

In [ ]:
dataset = pd.DataFrame(dados)
dataset.to_csv('.../nova_base_vagas.csv', index=False)

In [ ]:
dataset.head()

,Empresa,Vaga,Local,Nota,Link,Descrição,Sede,Tamanho,Fundação,Tipo,Receita,Oportunidades de carreira,Remuneração e benefícios,Cultura e valores,Alta liderança,Qualidade de vida,Recomendam para outras pessoas,Setor
0,Unisoma Computação,Analista de Data Science / Cientista de Dados,Campinas,3.5,https://www.glassdoor.com.br/partner/jobListin...,Oportunidade para quem tem conhecimento em mod...,"Campinas, Brasil",De 1 a 50 funcionários,1984,Empresa – capital fechado,De US$ 5 a US$ 25 milhões,2.7,3.5,3.7,3.5,4.0,74 %,NaN
1,Digio,Cientista de Dados Sênior,Trabalho remoto,4.4,https://www.glassdoor.com.br/partner/jobListin...,O objetivo da célula de Data Science do Digio ...,"Barueri, Brasil",De 201 a 500 funcionários,2013,Empresa – capital fechado,Desconhecido/Não se aplica,4.3,4.4,4.4,3.8,4.3,95 %,Financeiro
2,GRUPO AGROMERCANTIL,Líder Cientista de Dados,Uberlândia,5.0,https://www.glassdoor.com.br/partner/jobListin...,MISSÃO\nLiderar pessoas e projetos desenvolven...,"Uberlândia, Brasil",De 1 a 50 funcionários,NaN,Empresa – capital fechado,Desconhecido/Não se aplica,5.0,3.0,5.0,4.0,4.5,100 %,NaN
3,A3Data,Cientista de Dados Pleno e Sênior,Trabalho remoto,4.7,https://www.glassdoor.com.br/partner/jobListin...,A A3Data é uma consultoria referência em #Dado...,"Belo Horizonte, Brasil",De 51 a 200 funcionários,2014,Empresa – capital fechado,Desconhecido/Não se aplica,4.4,3.6,4.8,4.7,4.3,98 %,Tecnologia da informação
4,marketdata,Cientista de Dados Pleno,São Paulo,4.5,https://www.glassdoor.com.br/partner/jobListin...,"Quem Somos:\n\nA Marketdata, empresa do grupo ...","São Paulo, Brasil",De 201 a 500 funcionários,NaN,Subsidiária ou segmento de negócio,Desconhecido/Não se aplica,4.1,3.7,4.7,4.4,4.3,97 %,NaN
